In [1]:

import sys 
sys.path.append(r'C:\Users\sergiojesus\Desktop\Diogo2017\EQS_LOCAL\Bitbucket_eqs')
import analytics_stat.distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import traitlets
from tkinter import Tk, filedialog

display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
This report hides the raw code used for its implementation by default.
To hide/show the raw code, click <a href="javascript:code_toggle()">here</a>.'''))

display(HTML('''
<button id="do_run_all">Click to start</button>
<script>
$("#do_run_all").click(
    function () {
        $("#run_cell").click();
    }
);
</script>'''))
        

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
from spacy.language import Language
import re

nlp = Language(Vocab()) #Spacy variables
tokenizer = Tokenizer(nlp.vocab)
def tokenization_process(string):
    """tokenizes one string, removing not used punctuation in portuguese language in the process"""
    aux = tokenizer(string)
    output = list()
    for index, word in enumerate(aux):
        if re.search('([A-Za-zÀ-ÿ]+(-|\.)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)):
                output.append(str(re.search('([A-Za-zÀ-ÿ]+(-|\.)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower())
    return output

def toptokens(Tfidfmatrix, tokens, n=20):
    npmatrix = Tfidfmatrix.toarray()
    npmatrix[npmatrix == 0] = np.nan
    npmeans = np.nanmean(npmatrix, axis = 0)
    ordered_ids = np.argsort(npmeans)[::-1] 
    clean_ids = ordered_ids[(np.isnan(npmeans).sum()):]
    top_ids = clean_ids[:n]
    toptoken = [(tokens[i], npmeans[i]) for i in top_ids]
    df = pd.DataFrame(toptoken)
    df.columns = ['feature', 'TF-IDF']
    return df

In [3]:
class Display:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_excel)
        
        
        # Load Excel button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load File"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_excel)
        self._blj.disabled = True
                
        # Spreadsheet Dropdown
        style = {'description_width': 'initial'}
        self._dds = widgets.Dropdown(options=['Select file'], value='Select file', disabled=True, style=style,
                                     description='Spreadsheet:')
        
        # Column Selection
        self._csm = widgets.Select(options=['Select table'], value ='Select table', disabled=True)
        
        # Column Selection button
        self._bcs = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bcs.description = "Choose Columns"
        self._bcs.on_click(self.choose_columns)
        self._bcs.disabled = True
        
        # define layout
        box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='900px')
        
        self._box = widgets.Box(children=[self._bsf, self._dds, self._blj,
                                          self._csm, self._bcs], layout=box_layout, main_size = 10)
        display(self._box)
            
    def select_excel(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("XLS",'*.xls')])
        if new_file != "":
            self._bsf.files = new_file
            self._bsf.description = "File Selected"
            self._bsf.icon = "check-square-o"
            self._bsf.button_style = "success"
            self._blj.disabled = False
            print('Selected file: {}'.format(self._bsf.files))
            file_path = self._bsf.files
            self._excel = pd.ExcelFile(file_path[0], on_demand = True)
            self._sheets = self._excel.sheet_names
            self._dds.options = self._sheets
            self._dds.disabled = False

    def load_excel(self, b):
        self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value) 
        self._blj.button_style = "success"
        display(self._dataframe.head())
        self._csm.disabled = False
        self._csm.options = list(self._dataframe.columns.values)
        self._csm.disabled = False
        self._bcs.disabled = False
    def choose_columns(self, b):
        self._dataframe[self._csm.value].fillna("Empty Comment", inplace=True)
        self._documents = self._dataframe[self._csm.value]
        print(self._documents)

In [4]:
d = Display()

Box(children=(Button(description='Select File', icon='square-o', layout=Layout(height='50px', width='500px'), …

Selected file: ["C:/Users/sergiojesus/Desktop/Recursos/Dados/Incidentes e OT's - Pontes 114 a 146.2 MX6 e MX7.xls"]


,OT,Descrição OT,Relevante,recordkey,worklogid,Column1,Descrição Log,Descrição Longa Log,Estado,Tipo Trabalho,...,Activo,Descrição Ativo,Data Registo,Ano,Mês,Sector,Inío Intervenção,Reposição do Activo,Fim Intervenção,Incidendia
0,ALS100011,Ponte 144 em avaria e com falha de autolevel,n,ALS100011,25902.0,25902.0,NaN,Foi verificada a UPS e no local está sem alarm...,FECHADA,MC,...,32124V010020,Ponte Telescópica FP-TEK 144,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 07:35:00.000,2010-07-26 07:50:00.000,2010-07-26 07:50:00.000,1014.0
1,ALS100048,varia da manga 146,n,ALS100048,21676.0,21676.0,safty edge actuado. recolocada a manga ficando...,Depois de verificações a varios sistemas de se...,FECHADA,MC,...,32124V010022,Ponte Telescópica FP-TEK 146.1,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 18:15:00.000,2010-07-26 18:30:00.000,2010-07-26 18:32:00.000,1042.0
2,ALS100053,Avaria manga 143,n,ALS100053,21768.0,NaN,puxada a manga atrás e retirado o alarme fican...,NaN,FECHADA,MC,...,32124V010019,Ponte Telescópica FP-TEK 143,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 07:33:00.000,2010-07-27 07:45:00.000,2010-07-27 07:45:00.000,1059.0
3,ALS1000715,Reporte de ponte 146.2 fora de parque,n,ALS1000715,179442.0,179442.0,Reporte de ponte 146.2 fora de parque,Foram verificadas as duas pontes 146.1 e146.2 ...,REALIZ,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2012-11-21,2012,11,ALSCMT-PSE,2012-11-21 07:22:00.000,NaN,2012-11-21 07:32:00.000,42300.0
4,ALS100100,Avaria Ponte Telescópica FP-TEK 146.2,n,ALS100100,21774.0,21774.0,o operador tinha dificuldade em manobrar a manga,foi dada alguma informação de como proceder fi...,FECHADA,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 12:00:00.000,2010-07-27 12:20:00.000,2010-07-27 12:20:00.000,1073.0


0            Ponte 144 em avaria e com falha de autolevel
1                                      varia da manga 146
2                                        Avaria manga 143
3                   Reporte de ponte 146.2 fora de parque
4                   Avaria Ponte Telescópica FP-TEK 146.2
5                   Avaria Ponte Telescópica FP-TEK 146.2
6                               ALARME GERAL NA MANGA 145
7                               ALARME GERAL NA PONTE 142
8                               ALARME GERAL NA PONTE 142
9                                 Manga 146  com anomalia
10                    Camara da manga virada ao contrario
11                    Camara da manga virada ao contrario
12                    Camara da manga virada ao contrario
13                        Manga 115 - Falha de Auto Level
14      Ponte 146.2 ao finalizar acostamento em manual...
15                         PMR SPLITS MANGAS TELESCÓPICAS
16                                    Ponte 122 em avaria
17            

In [18]:
TFIDF = TfidfVectorizer(tokenizer=tokenization_process, use_idf=False)
counter = CountVectorizer(tokenizer=tokenization_process)
TFIDF_matrix = TFIDF.fit_transform(d._documents)
count_matrix = counter.fit_transform(d._documents)
terms = TFIDF.get_feature_names()
#tokens = toptokens(count_matrix, terms)

In [36]:
token_frequency = count_matrix.sum(0)

In [139]:
print(token_frequency.shape)
x = (token_frequency<=4).A1
trues = np.where(x)[0].tolist()
terms = np.array(terms)
terms[trues]

(1, 1643)


array(['a.c', 'a.g', 'aaf', ..., 'águas', 'ás', 'ãvaria'], dtype='<U17')

In [98]:
terms = counter.get_feature_names()
terms[trues]

TypeError: list indices must be integers or slices, not list

In [96]:
count_matrix.shape[1]

1643

In [149]:
def create_dict(count_matrix, terms, repeats=3):
    if len(terms) != count_matrix.shape[1]:
        raise ValueError("Terms and Count Matrix are not a match.")
    else:
        token_frequency = count_matrix.sum(0)
        repeated_words = terms[(token_frequency>repeats).A1]
        dubious_words = terms[(token_frequency<=repeats).A1]
        inexistent_words = []
        for term in dubious_words:
            if term in vec_dicionario:
                print(term)
            else:
                inexistent_words.append(term)
        print(inexistent_words)

In [101]:
dicionario = pd.DataFrame.from_csv('Dicionario.csv')

In [125]:
'a' in dicionario['Words']

False

In [112]:
dicionario['index'] = [i for i in range(63809 )]

In [124]:
 dicionario.reset_index(inplace = True)

In [130]:
something = ["roda"]
if dicionario['Words'].item(something):
    print(1)
else:
    print(0)

TypeError: item() takes 1 positional argument but 2 were given

In [138]:
'escadas' in np.array(dicionario['Words'])

True

In [140]:
vec_dicionario = np.array(dicionario['Words'])

In [150]:
create_dict(count_matrix, terms)

a.c
abanar
abandonada
abertas
abertura
abriu
acentuado
acidentes
acontecer
acoplada
acoplamento
acoplar
actua
actuar
actuação
acumulada
acumulação
afastado
aferição
aia
ainda
air
algum
alguma
alguns
alterar
amarela
ambas
ana
and
angulo
anormal
ante
apagada
apagado
apareceu
aparente
aparentemente
apenas
apesar
apresentar
aquando
aquecimento
area
arranca
arrancada
arrancado
arrancou
arrastar
assentou
assim
assume
assumiu
atingida
atingiu
atraso
autenticação
autorização
autómato
avança
avançava
avançou
avariado
aviso
baixar
baixe
balastro
banda
base
bastante
batente
bateu
beneficiação
bloquear
bolsa
botoneira
branco
braços
bride
buscar
c
cabeça
cabina
calha
calhas
camera
cancelamento
cargo
carregada
cartões
cedo
cedência
central
chegado
cheio
chuva
chão
cilindro
cima
cobertura
colado
colega
colocar
coluna
comandar
começou
completa
completam
completamente
completo
comportamento
confirmar
confirmação
conjunto
connect
conseguem
consegui
conseguimos
conseguindo
conseguir
conseguiram
constante

['a.g', 'aaf', 'aalrme', 'aarme', 'abanou', 'abruptamente', 'abrutamente', 'aceesso', 'acom', 'acompahamento', 'acomulação', 'acopstagem', 'acosta', 'acostamentos', 'acostei', 'activada', 'activando', 'activated', 'activação', 'activi', 'actuadops', 'actuados', 'actuou', 'aderencia', 'adge', 'aea', 'aeronane', 'aerone', 'aeroneve', 'aertonave', 'afeto', 'afr', 'afrente', 'afunda-se', 'agravada', 'aguarda', 'aircaft', 'aircraft', 'aklarme', 'alaeme', 'alame', 'alamre', 'alare', 'alareme', 'alarems', 'alarm', 'alarmegeral', 'alarmme', 'alarmne', 'alarrme', 'alatme', 'alerme', 'alimentção', 'alrarme', 'alrmes', 'alsaer', 'am', 'amnga', 'analise', 'angulos', 'anom', 'anomalos', 'antepara', 'aoa', 'aomalia', 'apôs', 'ar-condicionado', 'arme', 'arnadura', 'aslarme', 'assistencia', 'atingidos', 'ativada', 'ativo', 'ativos', 'atuada', 'atuados', 'aui', 'aujto', 'aurtolevel', 'autolel', 'automatica', 'autonivel', 'autonivelamneto', 'autonivelemanto', 'autonivelemento', 'autonuivelamento', 'auto